In [91]:
import pandas as pd
import numpy as np
import folium
from pyproj import Transformer
import geopandas as gpd
from shapely.geometry import Point


In [92]:
# 데이터 불러오기
data = pd.read_csv('/Users/joomacbook/Desktop/경남 빅데이터 공모전/산청하동_XY좌표계_생활인구집계_행정동코드.csv')

In [93]:
# 1. 전체 경남 행정구역 경계 불러오기 (GeoDataFrame)
# 예시 파일 경로: '경남_행정경계.shp'
gdf_admin = gpd.read_file('/Users/joomacbook/Desktop/경남 빅데이터 공모전/LARD_ADM_SECT_SGG_경남/LARD_ADM_SECT_SGG_48_202505.shp')

## 그리드

In [94]:
# 좌표 변환 설정
transformer = Transformer.from_proj(
    "+proj=tmerc +lat_0=38 +lon_0=128 +k=0.9999 +x_0=400000 +y_0=600000 "
    "+ellps=bessel +units=m +no_defs +towgs84=-115.80,474.99,674.11,1.16,-2.31,-1.63,6.43",
    "EPSG:4326", always_xy=True
    )
data[['longitude', 'latitude']] = data[['X', 'Y']].apply(
    lambda row: transformer.transform(row['X'], row['Y']), axis=1, result_type="expand"
)


In [95]:
# 격자 크기 설정 (500m x 500m)
grid_width, grid_height = 500, 500

def create_grids_and_aggregate(data, grid_width, grid_height):
    x_min, x_max = data['X'].min(), data['X'].max()
    y_min, y_max = data['Y'].min(), data['Y'].max()
    
    x_bins = np.arange(x_min, x_max + grid_width, grid_width)
    y_bins = np.arange(y_min, y_max + grid_height, grid_height)
    
    grid_data = []
    
    for i in range(len(x_bins) - 1):
        for j in range(len(y_bins) - 1):
            x_start, x_end = x_bins[i], x_bins[i + 1]
            y_start, y_end = y_bins[j], y_bins[j + 1]

            cell_data = data[
                (data['X'] >= x_start) & (data['X'] < x_end) &
                (data['Y'] >= y_start) & (data['Y'] < y_end)
            ]
            집계값 = cell_data['집계값'].sum()
            center_x, center_y = (x_start + x_end) / 2, (y_start + y_end) / 2
            center_lon, center_lat = transformer.transform(center_x, center_y)

            grid_data.append({
                'x_start': x_start, 'x_end': x_end,
                'y_start': y_start, 'y_end': y_end,
                '집계값': 집계값,
                'center_lat': center_lat, 'center_lon': center_lon
            })
    
    grid_df = pd.DataFrame(grid_data)
    grid_df = grid_df[grid_df['집계값'] > 0]  # 0이 아닌 셀만 남김
    grid_df = grid_df.sort_values(by='집계값', ascending=False)
    
    quantiles = np.array_split(grid_df.index, 5)
    
    def get_color(index):
        if index in quantiles[0]:
            return '#000080'
        elif index in quantiles[1]:
            return '#4169E1'
        elif index in quantiles[2]:
            return '#1E90FF'
        elif index in quantiles[3]:
            return '#87CEEB'
        else:
            return '#E0FFFF'
    
    grid_df['color'] = grid_df.index.map(get_color)
    
    return grid_df

grid_data = create_grids_and_aggregate(data, grid_width, grid_height)

def create_map(grid_data, output_file):
    m = folium.Map(location=[grid_data['center_lat'].mean(), grid_data['center_lon'].mean()], zoom_start=12)
    lat_ratio, lon_ratio = 0.00455, 0.00555
    
    for _, row in grid_data.iterrows():
        bounds = [
            [row['center_lat'] - (lat_ratio / 2), row['center_lon'] - (lon_ratio / 2)],
            [row['center_lat'] - (lat_ratio / 2), row['center_lon'] + (lon_ratio / 2)],
            [row['center_lat'] + (lat_ratio / 2), row['center_lon'] + (lon_ratio / 2)],
            [row['center_lat'] + (lat_ratio / 2), row['center_lon'] - (lon_ratio / 2)],
            [row['center_lat'] - (lat_ratio / 2), row['center_lon'] - (lon_ratio / 2)]
        ]
        folium.Polygon(
            locations=bounds,
            color=None, fill=True,
            fill_color=row['color'],
            fill_opacity=1.0
        ).add_to(m)
    
    m.save(output_file)

output_file = r"/Users/joomacbook/Desktop/경남 빅데이터 공모전/map_with_grid.html"
create_map(grid_data, output_file)
print(f"작업 완료! HTML 파일 생성됨: {output_file}")


작업 완료! HTML 파일 생성됨: /Users/joomacbook/Desktop/경남 빅데이터 공모전/map_with_grid.html


## 그리드 + 버퍼 + 경계

In [96]:
gdf = pd.read_csv('/Users/joomacbook/Desktop/경남 빅데이터 공모전/산청+하동_5년간산불발생좌표.csv')

In [97]:
gdf.drop('field1', axis=1, inplace=True)

In [ ]:
new_columns = [
    'damagearea', 'endday', 'endmonth', 'endtime', 'endyear', 'firecause',
    'locbunji', 'locdong', 'locgungu', 'locsi', 'startday', 'startdayofweek',
    'startmonth', 'starttime', 'startyear', 'locmenu', 'full_location'
]
gdf.columns = new_columns + ['GC_TYPE', 'CLEANADDR', 'Y', 'X']

In [99]:
# 좌표 → Point 형식으로 변환
geometry = [Point(xy) for xy in zip(gdf['X'], gdf['Y'])]

# GeoDataFrame 생성 (위경도 좌표계: EPSG:4326)
fire_gdf = gpd.GeoDataFrame(gdf, geometry=geometry, crs='EPSG:4326')


In [100]:
# 3. 버퍼 생성을 위해 meter 좌표계로 변환
fire_gdf_meter = fire_gdf.to_crs(epsg=5181)  # Korea TM

# 4. 1km, 3km, 5km 버퍼 생성
buffer_1km = fire_gdf_meter.buffer(1000).to_crs(epsg=4326)
buffer_2km = fire_gdf_meter.buffer(2000).to_crs(epsg=4326)
buffer_3km = fire_gdf_meter.buffer(3000).to_crs(epsg=4326)


In [101]:
gdf_admin = gpd.read_file('/Users/joomacbook/Desktop/경남 빅데이터 공모전/LARD_ADM_SECT_SGG_경남/LARD_ADM_SECT_SGG_48_202505.shp')

In [102]:
def create_map(grid_data, output_file, gdf_admin, fire_gdf):
    # 지도 생성 중심 좌표: 격자 데이터 기준
    m = folium.Map(location=[grid_data['center_lat'].mean(), grid_data['center_lon'].mean()], zoom_start=10)
    lat_ratio, lon_ratio = 0.00455, 0.00555

    # 1. 격자 시각화
    for _, row in grid_data.iterrows():
        bounds = [
            [row['center_lat'] - (lat_ratio / 2), row['center_lon'] - (lon_ratio / 2)],
            [row['center_lat'] - (lat_ratio / 2), row['center_lon'] + (lon_ratio / 2)],
            [row['center_lat'] + (lat_ratio / 2), row['center_lon'] + (lon_ratio / 2)],
            [row['center_lat'] + (lat_ratio / 2), row['center_lon'] - (lon_ratio / 2)],
            [row['center_lat'] - (lat_ratio / 2), row['center_lon'] - (lon_ratio / 2)]
        ]
        folium.Polygon(
            locations=bounds,
            color=None, fill=True,
            fill_color=row['color'],
            fill_opacity=1.0
        ).add_to(m)

    # 2. 산불 지점 버퍼 추가 (1km, 2km, 3km)
    buffer_distances = [1000, 2000, 3000]  # meters
    colors = ['yellow', 'orange', 'red']
    labels = ['1km', '2km', '3km']
    
    for dist, color, label in zip(buffer_distances, colors, labels):
        buffered = fire_gdf.copy()
        buffered = buffered.to_crs(epsg=5186)  # 거리 기반 버퍼 생성
        buffered['geometry'] = buffered.buffer(dist)
        buffered = buffered.to_crs(epsg=4326)  # 다시 지도 좌표계로

        folium.GeoJson(
            buffered,
            style_function=lambda x, col=color: {
                'color': col,
                'weight': 2,
                'fillOpacity': 0.1
            },
            name=f'{label} 버퍼'
        ).add_to(m)

    # 3. 하동군·산청군 행정경계선 추가
    gdf_boundary = gdf_admin[gdf_admin['SGG_NM'].str.contains('하동군|산청군')].to_crs(epsg=4326)
    folium.GeoJson(
        gdf_boundary,
        style_function=lambda x: {
            'color': 'green',
            'weight': 3,
            'fillOpacity': 0,
            'dashArray': '5, 5'
        },
        name='행정경계선'
    ).add_to(m)

    # 4. Layer Control
    folium.LayerControl().add_to(m)

    # 5. 저장
    m.save(output_file)



In [103]:
output_file = "/Users/joomacbook/Desktop/경남 빅데이터 공모전/map_with_all.html"
create_map(grid_data, output_file, gdf_admin, fire_gdf)

## 그리드 + 경계

In [104]:
# 1. 지도 객체 생성 (격자 중심 기준)
m = folium.Map(location=[grid_data['center_lat'].mean(), grid_data['center_lon'].mean()], zoom_start=12)

# 2. 격자 히트맵 시각화
lat_ratio, lon_ratio = 0.00455, 0.00555

for _, row in grid_data.iterrows():
    bounds = [
        [row['center_lat'] - (lat_ratio / 2), row['center_lon'] - (lon_ratio / 2)],
        [row['center_lat'] - (lat_ratio / 2), row['center_lon'] + (lon_ratio / 2)],
        [row['center_lat'] + (lat_ratio / 2), row['center_lon'] + (lon_ratio / 2)],
        [row['center_lat'] + (lat_ratio / 2), row['center_lon'] - (lon_ratio / 2)],
        [row['center_lat'] - (lat_ratio / 2), row['center_lon'] - (lon_ratio / 2)]
    ]
    folium.Polygon(
        locations=bounds,
        color=None, fill=True,
        fill_color=row['color'],
        fill_opacity=1.0
    ).add_to(m)

# 3. 행정경계선 추가 (산청군·하동군)
gdf_boundary = gdf_admin[gdf_admin['SGG_NM'].str.contains('하동군|산청군')].to_crs(epsg=4326)

folium.GeoJson(
    gdf_boundary,
    style_function=lambda x: {
        'color': 'green',
        'weight': 3,
        'fillOpacity': 0,
        'dashArray': '5, 5'
    },
    name='행정경계선'
).add_to(m)

# 4. Layer Control 추가
folium.LayerControl().add_to(m)

# 5. 저장
m.save("map_with_grid_and_boundary.html")
print("저장 완료: map_with_grid_and_boundary.html")

저장 완료: map_with_grid_and_boundary.html


In [105]:
data

,Unnamed: 0,X,Y,집계값,ADMI_CD,longitude,latitude
0,0,361802.0,301193.0,1.257083,48850310.0,127.577773,35.309118
1,1,362252.0,301193.0,1.495085,48850310.0,127.582722,35.309135
2,2,362302.0,301193.0,1.495085,48850310.0,127.583271,35.309137
3,3,362452.0,301193.0,1.491356,48850310.0,127.584921,35.309142
4,4,362502.0,301193.0,1.491356,48850310.0,127.585471,35.309144
...,...,...,...,...,...,...,...
29870,29870,409302.0,306993.0,1.520000,48860390.0,128.100190,35.362089
29871,29871,409452.0,307343.0,1.270000,48860390.0,128.101844,35.365243
29872,29872,409452.0,307393.0,1.420000,48860390.0,128.101845,35.365694
29873,29873,409502.0,307493.0,1.020000,48860390.0,128.102396,35.366595
